In [ ]:
import math


# 画像認識（YOLOを含む）

画像認識（YOLOを含む）は、損失が下がる理由を『式』と『数値変化』の両方で確認する構成です。


## 概念の土台

画像認識（YOLOを含む）に入る前に、つまずきやすい用語を先にそろえます。以降のコードでは、変数がどの概念を表しているかを対応付けながら読んでください。

- **パラメータ θ**: モデルが学習で更新する重み集合です。予測の形を決めます。
- **順伝播**: 入力から出力へ値を流して予測を計算する過程です。
- **逆伝播**: 損失から勾配を計算して各パラメータの更新量を求める過程です。
- **バウンディングボックス**: 物体位置を矩形で表す予測形式です。
- **IoU**: 予測矩形と正解矩形の重なり率です。検出評価の基礎です。
- **NMS**: 重複検出を抑える後処理です。

このノートでは、ここで定義した語を実験セルの変数・式に直接対応させて確認します。


## 観察 1: YOLO出力テンソルの形

物体検出の実装で最初に混乱しやすいのは出力次元です。最小コードで形を固定して理解します。


In [ ]:
grid_h, grid_w, anchors, classes = 13, 13, 3, 80
channels = anchors * (5 + classes)
print('output shape =', (grid_h, grid_w, channels))
print('per_anchor_dim =', 5 + classes)
print('total_boxes =', grid_h * grid_w * anchors)
x = [0.8, -0.4, 0.2]
w = [0.3, -0.6, 0.5]
z = sum(xi * wi for xi, wi in zip(x, w)) + 0.1
y = 1 / (1 + math.exp(-z))
print('seed probability =', round(y, 4))

次元を先に理解すると、後続のデコード実装で起こるインデックスバグを大幅に減らせます。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 2: 損失を計算する

次に、予測値に対する損失を計算します。学習は損失を下げる方向に進むので、損失の意味を言葉で説明できることが重要です。


In [ ]:
target = 1.0
eps = 1e-9
loss = -(target * math.log(y + eps) + (1 - target) * math.log(1 - y + eps))
print('loss =', round(loss, 6))
print('error =', round(target - y, 6))

損失は『どれだけ外したか』を測る物差しです。物差しがない状態では、改善の方向を決められません。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 計算の対応表

1. $C_{out} = A\,(5 + C)$
2. $\mathrm{bbox} = (x,y,w,h) + o + p_{cls}$


## 観察 3: 勾配降下法の一歩

ここで、重みを一回更新する最小実験を行います。更新前後の損失を比較して、学習の方向が正しいかを確認します。


In [ ]:
lr = 0.1
grad = -(target - y) * y * (1 - y)
w_new = [wi - lr * grad * xi for wi, xi in zip(w, x)]
z_new = sum(xi * wi for xi, wi in zip(x, w_new)) + 0.1
y_new = 1 / (1 + math.exp(-z_new))
print('y before/after =', round(y, 6), round(y_new, 6))

この更新で損失が下がれば、勾配方向が合理的だったと言えます。下がらないなら学習率や符号を疑います。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 4: 正則化の感覚を作る

次に、重みが大きくなりすぎることを抑える正則化項を追加します。過学習対策をコードで体験するのが狙いです。


In [ ]:
l2 = 0.01
weight_norm = sum(wi * wi for wi in w_new)
loss_reg = loss + l2 * weight_norm
print('weight_norm =', round(weight_norm, 6))
print('regularized loss =', round(loss_reg, 6))

正則化は精度を上げる魔法ではなく、汎化の崩れを防ぐ保険です。評価データと合わせて効果を判断してください。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 5: ミニバッチの考え方

最後に、複数サンプルをまとめて扱う発想を確認します。実務では1件ずつよりバッチ処理が主流です。


In [ ]:
batch = [[0.8, -0.4, 0.2], [0.2, 0.1, -0.3], [0.5, -0.2, 0.7]]
targets = [1.0, 0.0, 1.0]
preds = []
for bx in batch:
    z_b = sum(xi * wi for xi, wi in zip(bx, w_new)) + 0.1
    preds.append(1 / (1 + math.exp(-z_b)))
print('preds =', [round(p, 4) for p in preds])

ミニバッチを扱えるようになると、計算効率と学習安定性の両面で設計の幅が広がります。

この節では、順伝播 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 要点整理

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 学習率が大きすぎて発散する
2. 検証損失の監視をせず過学習を見逃す
3. 前処理と活性化の相性を無視する
